### (Inspired from https://towardsdatascience.com/anomaly-detection-in-time-series-sensor-data-86fd52e62538)

In [ ]:
import sys
from os import listdir
from os.path import isfile, join
import datetime

import scipy as sy
import scipy.fftpack as syfp
from tqdm.notebook import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mtpl
import numpy as np

from pyapnea.oscar.oscar_loader import load_session
from pyapnea.oscar.oscar_getter import event_data_to_dataframe
from pyapnea.oscar.oscar_constants import CHANNELS, ChannelID

In [ ]:
# all files
data_path_cpap1 = '../data/raw/ResMed_23192565579/Events'
list_files = [{'label': f, 'value': f, 'fullpath': join(data_path_cpap1, f)} for f in listdir(data_path_cpap1) if isfile(join(data_path_cpap1, f))]
data_path_cpap2= '../data/raw/ResMed_23221085377/Events'
list_files.extend([{'label': f, 'value': f, 'fullpath': join(data_path_cpap2, f)} for f in listdir(data_path_cpap2) if isfile(join(data_path_cpap2, f))])

In [ ]:
sfreq = 25

In [ ]:
filename_to_load = '../data/raw/ResMed_23192565579/Events/62202198.001'
oscar_session_data = load_session(filename_to_load)
df = event_data_to_dataframe(oscar_session_data, [ChannelID.CPAP_FlowRate.value, ChannelID.CPAP_ClearAirway.value, ChannelID.CPAP_Obstructive.value])
df.set_index('time_utc', inplace=True)
df

In [ ]:
# reorganize dataframe to seaborn imput format

dfc = df[['FlowRate']]
# dfm = dfc.melt('time_utc', var_name='cols', value_name='vals')
# dfm.sort_values(by=['time_utc'], inplace=True, ignore_index=True)
# dfm_annotation = dfm[(~pd.isnull(dfm['vals']) & ((dfm['cols']=='Obstructive') | (dfm['cols']=='ClearAirway')))]
# display(dfm_annotation)
dfm_annotation=df[~df['Obstructive'].isna() | ~df['ClearAirway'].isna()]
dfm_annotation=dfm_annotation[['Obstructive', 'ClearAirway']]
dfm_annotation

## Exploratory Data Analysis (EDA)

In [ ]:

_ = plt.figure(figsize=(18,6))
_ = plt.plot(dfc['FlowRate'], color='blue')
_ = plt.plot(dfm_annotation[~dfm_annotation['Obstructive'].isna()], linestyle='none', marker='X', color='red', markersize=8)
_ = plt.plot(dfm_annotation[~dfm_annotation['ClearAirway'].isna()], linestyle='none', marker='X', color='green', markersize=8)
_ = plt.title('FlowRate')
plt.show()

In [ ]:
rollmean = dfc.resample(rule='4S').mean()
rollstd = dfc.resample(rule='4S').std()

In [ ]:
_ = plt.figure(figsize=(25,10))
_ = plt.plot(dfc['FlowRate'], color='blue', label='Original')
_ = plt.plot(rollmean['FlowRate'], color='red', label='Rolling Mean')
_ = plt.plot(rollstd['FlowRate'], color='black', label='Rolling Std' )
_ = plt.plot(dfm_annotation[~dfm_annotation['Obstructive'].isna()], linestyle='none', marker='X', color='red', markersize=10)
_ = plt.plot(dfm_annotation[~dfm_annotation['ClearAirway'].isna()], linestyle='none', marker='X', color='green', markersize=10)
_ = plt.legend(loc='best')
_ = plt.title('FlowRate')
plt.show()

In [ ]:
print(len(dfc['FlowRate']))
_ = plt.figure(figsize=(25,10))
_ = plt.specgram(dfc['FlowRate'], Fs=25, NFFT=64, noverlap=2, cmap='nipy_spectral')
plt.xlabel('Time (s)')
plt.ylabel('Frequency (Hz)')
plt.colorbar(label='Intensity (dB)')
plt.show()

In [ ]:
# from https://stackoverflow.com/questions/26105698/how-to-calculate-frequency-of-a-give-wave-and-time
# Do FFT analysis of array
dt = 40/1000  # 40 ms
FFT = sy.fft.fft(dfc['FlowRate'].values)

# Getting the related frequencies
freqs = syfp.fftfreq(len(dfc['FlowRate']), dt)

_ = plt.figure(figsize=(25,10))
_ = plt.plot(freqs, np.lib.scimath.log10(abs(FFT)), '.')
plt.xlim(-1, 1)
plt.show()

In [ ]:
# frequency most common in the signal
from operator import itemgetter
index, element = max(enumerate(np.lib.scimath.log10(abs(FFT))), key=itemgetter(1))
common_freq = freqs[index]
print('freq (Hz) = ', common_freq)
# most common breath length (s)
most_common_breath_length = 1/common_freq
print('most common breath len (s)', most_common_breath_length)

In [ ]:
event_time = dfm_annotation.index[1]
_ = plt.figure(figsize=(25,10))
_ = plt.plot(dfc['FlowRate'], color='blue', label='Original')
_ = plt.plot(rollmean['FlowRate'], color='red', label='Rolling Mean')
_ = plt.plot(rollstd['FlowRate'], color='black', label='Rolling Std')
_ = plt.plot(dfm_annotation[~dfm_annotation['Obstructive'].isna()], linestyle='none', marker='X', color='red', markersize=10)
_ = plt.plot(dfm_annotation[~dfm_annotation['ClearAirway'].isna()], linestyle='none', marker='X', color='green', markersize=10)
_ = plt.legend(loc='best')
_ = plt.title('FlowRate')
plt.ylim(-50, 50)
plt.xlim(event_time - datetime.timedelta(seconds=50), event_time + datetime.timedelta(seconds=10))
plt.show()

In [ ]:
print(len(dfc['FlowRate'].loc[event_time - datetime.timedelta(seconds=50): event_time + datetime.timedelta(seconds=10)]))
_ = plt.figure(figsize=(25,10))
_ = plt.specgram(dfc['FlowRate'].loc[event_time - datetime.timedelta(seconds=50): event_time + datetime.timedelta(seconds=10)], Fs=25, NFFT=16, noverlap=2, cmap='nipy_spectral')
plt.xlabel('Time (s)')
plt.ylabel('Frequency (Hz)')
plt.colorbar(label='Intensity (dB)')
plt.show()

## stationarity and autocorrelation
see https://statisticsbyjim.com/time-series/autocorrelation-partial-autocorrelation/
https://analyzingalpha.com/time-series-analysis-with-python

In [ ]:
from statsmodels.tsa.stattools import adfuller
# Run Augmented Dickey Fuller Test
result = adfuller(dfc['FlowRate'], autolag=None, maxlag=100)
# Print p-value
print(result[1])

In [ ]:
# Plot ACF
from statsmodels.graphics.tsaplots import plot_acf
with mtpl.rc_context():
    mtpl.rc("figure", figsize=(20,15))
    plot_acf(dfc['FlowRate'].dropna(), lags=400, zero=False)
# See https://stackoverflow.com/questions/63189488/no-confidence-interval-are-shown-when-using-plot-acf for high-resolution time series

# Models

## Interquartile Range

Strategy:

    Calculate IQR which is the difference between 75th (Q3)and 25th (Q1) percentiles.
    Calculate upper and lower bounds for the outlier.
    Filter the data points that fall outside the upper and lower bounds and flag them as outliers.
    Finally, plot the outliers on top of the time series data (the readings from sensor_11 in this case)

In [ ]:
# Calculate IQR for the FlowRate component
df_iqr = dfc.copy()
q1_pc1, q3_pc1 = df_iqr['FlowRate'].quantile([0.25, 0.75])
iqr_pc1 = q3_pc1 - q1_pc1 # Calculate upper and lower bounds for outlier for pc1
lower_pc1 = q1_pc1 - (1.5*iqr_pc1)
upper_pc1 = q3_pc1 + (1.5*iqr_pc1) # Filter out the outliers from the pc1
df_iqr['anomaly_pc1'] = ((df_iqr['FlowRate']>upper_pc1) | (df_iqr['FlowRate']<lower_pc1))
a = df_iqr[df_iqr['anomaly_pc1'] == 1] #anomaly
_ = plt.figure(figsize=(18,6))
_ = plt.plot(df_iqr['FlowRate'], color='blue', label='Normal')
_ = plt.plot(a['FlowRate'], linestyle='none', marker='X', color='purple', markersize=10, label='Anomaly')
_ = plt.plot(dfm_annotation[~dfm_annotation['Obstructive'].isna()], linestyle='none', marker='X', color='red', markersize=10)
_ = plt.plot(dfm_annotation[~dfm_annotation['ClearAirway'].isna()], linestyle='none', marker='X', color='green', markersize=10)
_ = plt.xlabel('Date and Time')
_ = plt.ylabel('Sensor Reading')
_ = plt.title('FlowRate Anomalies')
_ = plt.legend(loc='best')
plt.show();

## K-Means Clustering

Strategy:

    Calculate the distance between each point and its nearest centroid. The biggest distances are considered as anomaly.
    We use outliers_fraction to provide information to the algorithm about the proportion of the outliers present in our data set. Situations may vary from data set to data set. However, as a starting figure, I estimate outliers_fraction=0.13 (13% of df are outliers as depicted).
    Calculate number_of_outliers using outliers_fraction.
    Set threshold as the minimum distance of these outliers.
    The anomaly result of anomaly1 contains the above method Cluster (0:normal, 1:anomaly).
    Visualize anomalies with Time Series view.

In [ ]:
# Import necessary libraries
from sklearn.cluster import KMeans
df_km = dfc.copy()
kmeans = KMeans(n_clusters=2, random_state=42)
kmeans.fit(df_km.values)
labels = kmeans.predict(df_km.values)
unique_elements, counts_elements = np.unique(labels, return_counts=True)
clusters = np.asarray((unique_elements, counts_elements))# Write a function that calculates distance between each point and the centroid of the closest cluster

_ = plt.figure(figsize = (9, 7))
_ = plt.bar(clusters[0], clusters[1], tick_label=clusters[0])
_ = plt.xlabel('Clusters')
_ = plt.ylabel('Number of points')
_ = plt.title('Number of points in each cluster')
plt.show()

In [ ]:
_ = plt.figure(figsize=(9,7))
_ = plt.scatter(df_km['FlowRate'], df_km['FlowRate'], c=labels)
_ = plt.xlabel('FlowRate')
_ = plt.ylabel('FlowRate')
_ = plt.title('K-means of clustering')
plt.show()

In [ ]:
def getDistanceByPoint(data, model):
    """ Function that calculates the distance between a point and centroid of a cluster,
            returns the distances in pandas series"""
    distance = []
    for i in range(0,len(data)):
        Xa = np.array(data.iloc[i])
        Xb = model.cluster_centers_[model.labels_[i]-1]
        distance.append(np.linalg.norm(Xa-Xb))
    return pd.Series(distance, index=data.index) # Assume that 13% of the entire data set are anomalies

outliers_fraction = 0.01# get the distance between each point and its nearest centroid. The biggest distances are considered as anomaly
distance = getDistanceByPoint(df_km, kmeans)# number of observations that equate to the 13% of the entire data set
number_of_outliers = int(outliers_fraction*len(distance))# Take the minimum of the largest 13% of the distances as the threshold
threshold = distance.nlargest(number_of_outliers).min()# anomaly1 contain the anomaly result of the above method Cluster (0:normal, 1:anomaly)
df_km['anomaly1'] = (distance >= threshold)

df_km.head()

In [ ]:
df_km['anomaly1'].value_counts()

In [ ]:
#df_km['anomaly1'] = pd.Series(df_km['anomaly1'].values, index=df_km.index)
a = df_km[df_km['anomaly1'] == 1] #anomaly
_ = plt.figure(figsize=(18,6))
_ = plt.plot(df_km['FlowRate'], color='blue', label='Normal')
_ = plt.plot(a['FlowRate'], linestyle='none', marker='X', color='purple', markersize=10, label='Anomaly')
_ = plt.plot(dfm_annotation[~dfm_annotation['Obstructive'].isna()], linestyle='none', marker='X', color='red', markersize=10)
_ = plt.plot(dfm_annotation[~dfm_annotation['ClearAirway'].isna()], linestyle='none', marker='X', color='green', markersize=10)
_ = plt.xlabel('Date and Time')
_ = plt.ylabel('Sensor Reading')
_ = plt.title('FlowRate Anomalies')
_ = plt.legend(loc='best')
plt.show();


## Isolation Forest
Aucun sens sur des données 1D : https://stackoverflow.com/questions/50957340/isolation-forest-sklearn-for-1d-array-or-list-and-how-to-tune-hyper-parameters

In [ ]:
# Import IsolationForest
from sklearn.ensemble import IsolationForest
df_if = dfc.copy()

# Assume that 13% of the entire data set are anomalies
outliers_fraction = 0.13
model =  IsolationForest(contamination=outliers_fraction, )
model.fit(df_if.values)
df_if['anomaly2'] = pd.Series(model.predict(df_if.values))
a = df_if.loc[df_if['anomaly2'] == -1] #anomaly
a.head()

In [ ]:
# visualization
#df_if['anomaly2'] = pd.Series(df_if['anomaly2'].values, index=df.index)

_ = plt.figure(figsize=(18,6))
_ = plt.plot(df_if['FlowRate'], color='blue', label='Normal')
_ = plt.plot(a['FlowRate'], linestyle='none', marker='X', color='purple', markersize=10, label='Anomaly')
_ = plt.plot(dfm_annotation[~dfm_annotation['Obstructive'].isna()], linestyle='none', marker='X', color='red', markersize=10)
_ = plt.plot(dfm_annotation[~dfm_annotation['ClearAirway'].isna()], linestyle='none', marker='X', color='green', markersize=10)
_ = plt.xlabel('Date and Time')
_ = plt.ylabel('Sensor Reading')
_ = plt.title('FlowRate Anomalies')
_ = plt.legend(loc='best')
plt.show();